In [1]:
import sys
import os  
import traceback

RECORDS_PATH = '../input/tfrecords'

if os.path.exists('/kaggle'):
  sys.path.append('/kaggle/input/gezi-melt/utils/utils')
  sys.path.append('/kaggle/input/official')
  from kaggle_datasets import KaggleDatasets
  try:
    RECORDS_PATH = KaggleDatasets().get_gcs_path('toxic-multi-tfrecords') + '/tfrecords/tfrecords'
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
  except Exception:
    print(traceback.format_exc())
    RECORDS_PATH = '../input/toxic-multi-tfrecords/tfrecords/tfrecords'
    pass
!ls ../input
RECORDS_PATH

bert-multi					  tfrecords
huggingface					  tf-xlm-roberta-base
jigsaw-multilingual-toxic-comment-classification  tf-xlm-roberta-large


'../input/tfrecords'

In [2]:
import official
import gezi
import melt
import lele
import husky
import pandas as pd
import numpy as np
import tensorflow as tf
from absl import app, flags
FLAGS = flags.FLAGS
try:
    flags.DEFINE_string('model', None, '')
    flags.DEFINE_bool('multi_head', False, '')
    flags.DEFINE_string('pretrained', '../input/tf-xlm-roberta-large', '')
    flags.DEFINE_string('pretrained2', None, '')
    flags.DEFINE_integer('max_len', None, 'xlm 192 bert 128')
    flags.DEFINE_bool('freeze_pretrained', False, '')
    flags.DEFINE_bool('valid_en', False, '')
    flags.DEFINE_alias('ve', 'valid_en')
    flags.DEFINE_bool('test_en', None, '')
    flags.DEFINE_bool('use_lang', False, '')
    flags.DEFINE_bool('use_mlp', False, '')
    flags.DEFINE_bool('use_src', False, '')
    flags.DEFINE_bool('use_trans', False, '')
    flags.DEFINE_string('task', 'toxic', '')
except Exception:
    pass
tf.__version__

'2.1.0'

In [3]:
# flags
argv=['']
FLAGS(argv)
mark='xlm'
FLAGS.train_input=f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train'
FLAGS.valid_input=f'{RECORDS_PATH}/{mark}/validation'
FLAGS.test_input=f'{RECORDS_PATH}/{mark}/test'
FLAGS.valid_interval_steps=0
FLAGS.verbose=1 
FLAGS.num_epochs=1
FLAGS.keras=1 
FLAGS.buffer_size=2048
FLAGS.learning_rate=3e-5 
FLAGS.min_learning_rate=0.
# FLAGS.opt_epsilon=1e-8 
FLAGS.optimizer='bert-adamw'
# FLAGS.optimizer='adam'
FLAGS.metrics=['acc', 'auc'] 
FLAGS.test_names=['id', 'toxic']
FLAGS.valid_interval_epochs=0.1
FLAGS.test_interval_epochs=1.
FLAGS.num_gpus=8
FLAGS.cache=0
FLAGS.model_dir='../working/exps/unintended'
# FLAGS.ckpt_dir='../input/single-unintended'
FLAGS.multi_head=0
FLAGS.batch_parse=1
FLAGS.save_model=1
# FLAGS.pretrained = '../input/tf-xlm-roberta-large/'
FLAGS.pretrained = '../input/tf-xlm-roberta-base/'
FLAGS.batch_size=16 if 'large' in FLAGS.pretrained else 32
FLAGS.debug=0
FLAGS.sparse_to_dense=1 
FLAGS.padding_idx=1 
# FLAGS.buckets=190,350 
FLAGS.buckets=[]
FLAGS.batch_size=32 
# FLAGS.batch_sizes=32,16,8 if not 'large' in FLAGS.pretrained else 16,8,4
FLAGS.batch_sizes=[]
# FLAGS.length_key='input_word_ids'
FLAGS.max_len=192
FLAGS.do_test=0

toxic_types = ['severe_toxic', 'obscene', 'identity_hate', 'threat', 'insult']
langs = ['en', 'es', 'it', 'tr', 'fr', 'pt', 'ru']
srcs = ['unintended', 'toxic', 'test']

BERT_GCS_PATH_SAVEDMODEL = '../input/bert-multi/bert_multi_from_tfhub'
RECORDS_GCS_PATH = '../input/tfrecords'

def init():
  # if FLAGS.mode == 'valid' or FLAGS.mode == 'test':
  #   FLAGS.gpus = 1
  if FLAGS.valid_en:
    if FLAGS.test_en is None:
      FLAGS.test_en = True
    FLAGS.valid_input = FLAGS.valid_input.replace('validation', 'validation-en')
    FLAGS.train_input = FLAGS.train_input.replace('validation', 'validation-en')
  if FLAGS.test_en:
    FLAGS.test_input = FLAGS.test_input.replace('test', 'test-en')
  if 'bylang' in FLAGS.valid_input:
    if FLAGS.folds is not None:
      FLAGS.folds = 3
  
  if FLAGS.max_len:
    FLAGS.keras_loop = True
    FLAGS.sparse_to_dense = False
    
  if 'pair' in FLAGS.train_input:
    FLAGS.batch_size = int(FLAGS.batch_size / 2)
    if FLAGS.max_len:
      FLAGS.max_len *= 2

init()

In [4]:
# evaluate
from collections import OrderedDict
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.utils import shuffle

import gezi
logging = gezi.logging

eval_langs = ['es', 'it', 'tr']

def evaluate(y_true, y_pred, x):
  if FLAGS.task == 'toxic':
    try:
      y_true = y_true[:,0]
      y_pred = y_pred[:,0]
    except Exception:
      pass
    if y_pred.max() > 1. or y_pred.min() < 0:
      y_pred = gezi.sigmoid(y_pred)
    result = OrderedDict()
    loss = log_loss(y_true, y_pred)
    result['loss'] = loss
    
    auc = roc_auc_score(y_true, y_pred)
    result['auc/all'] = auc
      
    if 'lang' in x:
      x['y_true'] = y_true
      x['pred'] = y_pred
      x['lang'] = gezi.decode(x['lang'])

      df = pd.DataFrame(x) 
      df = shuffle(df)
      gezi.pprint(pd.concat([df[df.y_true==0].sample(5), df[df.y_true==1].sample(5)]), 
                  print_fn=logging.info, desc='preds:')

      for lang in eval_langs:
        df_ = df[df.lang==lang]
        if len(df_):
          auc = roc_auc_score(df_.y_true, df_.pred)
          result[f'auc/{lang}'] = auc
  elif FLAGS.task == 'lang':
    results = np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1)
    result = {'acc': np.sum(results) / len(results)}

  return result

In [5]:
# utils

def get_lang_id(x):
  return tf.cast(tf.math.equal(x, 'en'), tf.int32) \
         + tf.cast(tf.math.equal(x, 'es'), tf.int32) * 2 \
         + tf.cast(tf.math.equal(x, 'it'), tf.int32) * 3 \
         + tf.cast(tf.math.equal(x, 'tr'), tf.int32) * 4 \
         + tf.cast(tf.math.equal(x, 'fr'), tf.int32) * 5 \
         + tf.cast(tf.math.equal(x, 'pt'), tf.int32) * 6 \
         + tf.cast(tf.math.equal(x, 'ru'), tf.int32) * 7 \
         - 1

# srcs = ['unintended', 'toxic', 'test']
def get_src_id(x):
  return tf.cast(tf.math.equal(x, 'unintended'), tf.int32) \
         + tf.cast(tf.math.equal(x, 'toxic'), tf.int32) * 2 \
         + tf.cast(tf.math.equal(x, 'test'), tf.int32) * 3 \
         - 1

In [6]:
# dataset
import tensorflow as tf
from tensorflow.keras import backend as K
import melt

class Dataset(melt.Dataset):
  def __init__(self, subset='valid', **kwargs):
    super(Dataset, self).__init__(subset, **kwargs)

  def parse(self, example):
    MAX_LEN = FLAGS.max_len
    features_dict = {
      'toxic': tf.io.FixedLenFeature([], tf.float32),
      'id': tf.io.FixedLenFeature([], tf.string),
      # 'comment_text': tf.io.FixedLenFeature([], tf.string), # TODO
    }
    features_dict['input_word_ids'] = tf.io.VarLenFeature(tf.int64) if not MAX_LEN else  tf.io.FixedLenFeature([MAX_LEN], tf.int64)

    def _adds(names, dtype=None, length=None):
      dtype_ = dtype
      for name in names:
        if name in self.example:
          dtype = dtype_ or self.example[name].dtype 
          if length:
            features_dict[name] = tf.io.FixedLenFeature([length], dtype)
          else:
            features_dict[name] = tf.io.FixedLenFeature([], dtype)

    _adds(['lang', 'src'], tf.string)
    _adds(['trans'], tf.int64, 1)

    _adds(['input_mask', 'all_segment_id'], tf.int64, MAX_LEN)
    
    _adds(toxic_types)

    features = self.parse_(serialized=example, features=features_dict)

    features['lang_'] = tf.expand_dims(get_lang_id(features['lang']), -1)
    features['src_'] = tf.expand_dims(get_src_id(features['src']), -1)

    def _casts(names, dtype=tf.int32):
      for name in names:
        if name in features:
          features[name] = tf.cast(features[name], dtype)

    _casts(['input_word_ids', 'input_mask', 'all_segment_id', 'trans'])

    x = features

    if FLAGS.task == 'toxic':
      y = features['toxic']
    #     y = tf.cast(features['toxic'] > 0.5, tf.float32)
      keys = ['toxic', *toxic_types]
      for key in keys:
        if key not in features:
          features[key] = tf.zeros_like(features['toxic'])
          
      _casts(toxic_types, tf.float32)
          
      melt.append_dim(features, keys)

      if FLAGS.multi_head:
        y = tf.concat([features[key] for key in keys], 1)


    elif FLAGS.task == 'lang':
      y = tf.one_hot(features['lang_'], len(langs))

 
    # self.padding_values = (melt.get_padding_values(features, 1), 1.)

    return x, y
 

In [7]:
# loss
import tensorflow as tf

def calc_loss(y_true, y_pred):
  pass

def focal_loss(gamma=1.5, alpha=.2):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def get_loss_fn():
#   return tf.compat.v1.losses.sigmoid_cross_entropy
  return tf.keras.losses.BinaryCrossentropy()
#   return focal_loss()

In [8]:
from tensorflow.keras.layers import Input
from tensorflow import keras
from tensorflow.keras import backend as K

import transformers
from transformers import TFAutoModel

I0425 00:04:51.617229 140030697813824 file_utils.py:41] PyTorch version 1.4.0 available.
I0425 00:04:51.617975 140030697813824 file_utils.py:57] TensorFlow version 2.1.0 available.


In [9]:
# FLAGS.use_mlp=1
# FLAGS.use_lang=1
# FLAGS.use_src=1
# FLAGS.use_trans=1

In [10]:
class XlmModel(keras.Model):
  def __init__(self):
    super(XlmModel, self).__init__() 

    pretrained = FLAGS.pretrained

    with gezi.Timer(f'load xlm_model from {pretrained}', True, logging.info):
      self.transformer = TFAutoModel.from_pretrained(pretrained)

    # if FLAGS.pretrained2:
    #   logging.info(f'load lang_model from {FLAGS.pretrained2}')
    #   self.transformer.load_weights(FLAGS.pretrained2)

    if FLAGS.freeze_pretrained:
      self.transformer.trainable = False
    # dims = [32]
    # self.mlp = melt.layers.MLP(dims)
    if FLAGS.use_lang:
      self.lang_emb = keras.layers.Embedding(len(langs), 32, name='lang_emb')
    if FLAGS.use_src:
      self.src_emb = keras.layers.Embedding(len(srcs), 32, name='src_emb')
    if FLAGS.use_trans:
      self.trans_emb = keras.layers.Embedding(2, 32, name='trans_emb')
    
    if FLAGS.use_mlp:
      self.mlp = melt.layers.MLP([256,128])
    odim = len(toxic_types) + 1 if FLAGS.multi_head else 1
    self.dense = keras.layers.Dense(odim, activation='sigmoid')

  def call(self, input):
    input_word_ids = input['input_word_ids']
    # tf.print(input_word_ids)
    # tf.print(input['id'])
    # tf.print(input['toxic'])
    x = self.transformer(input_word_ids)[0]
    x = x[:, 0, :]
    
    others = []
    if FLAGS.use_lang:
      lang_ids = input['lang_']
      x_lang = tf.squeeze(self.lang_emb(lang_ids), 1)
      others += [x_lang]
  
    if FLAGS.use_src:
      srcs = input['src_']
      x_src = tf.squeeze(self.src_emb(srcs), 1)
      others += [x_src]
        
    if FLAGS.use_trans:
      trans = input['trans']
      x_trans = tf.squeeze(self.trans_emb(trans), 1)
      others += [x_trans]
    
    if others:
      x = tf.concat([x, *others], -1)
      
    if FLAGS.use_mlp:
      x = self.mlp(x)
    x = self.dense(x)
    # tf.print(x)
    return x


def xlm_model():
  inputs = {
            'input_word_ids': Input(shape=(FLAGS.max_len,), dtype=tf.int32, name="input_word_ids"),
            'lang_': Input(shape=(1,), dtype=tf.int32, name="lang_"),
            'src_': Input(shape=(1,), dtype=tf.int32, name="src_"),
            'trans': Input(shape=(1,), dtype=tf.int32, name="trans"),
            }
  out = XlmModel()(inputs)

  model = keras.Model(inputs=list(inputs.values()), outputs=out)

  return model

In [11]:
Model = xlm_model

In [12]:
# train

import os
import melt

fit = melt.fit
melt.init()
loss_fn = get_loss_fn()

2020-04-25 00:04:51 0:00:00 fcntl.floc with lock_file /root/.melt.lock (If hang here means other programs calling melt.init have not finished yet)
2020-04-25 00:04:52 0:00:00 Tf dataset and Tf model train in Eager mode, keras 1, distributed:False
2020-04-25 00:04:52 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-04-25 00:04:52 0:00:00 batch_size: 256 eval_batch_size: 256 batch_size_per_gpu: 32 num_gpus: 8 gpu: [0, 1, 2, 3, 4, 5, 6, 7] CUDA_VISIABLE_DEVICES=[] work_mode: train distributed: False horovod: False
2020-04-25 00:04:57 0:00:05 model: [unintended] model_dir: [../working/exps/unintended]


In [13]:
strategy = melt.distributed.get_strategy()
with strategy.scope():
  model = Model()

2020-04-25 00:04:57 0:00:05 load xlm_model from ../input/tf-xlm-roberta-base/ start
I0425 00:04:57.795662 140030697813824 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-base/config.json
I0425 00:04:57.796745 140030697813824 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_em

In [14]:
def run(model=model):
  fit(model,  
      loss_fn,
      Dataset,
      eval_fn=evaluate,
      eval_keys=['lang'],
      )   

In [15]:
init_weights = model.get_weights()

In [16]:
# FLAGS.train_input=','.join([f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train',
#                            f'{RECORDS_PATH}/{mark}/jigsaw-unintended-bias-train',
#                            f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train-google-tr-cleaned',
#                            f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train-google-it-cleaned',
#                            f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train-google-es-cleaned',
#                            f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train-google-ru-cleaned',
#                            f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train-google-pt-cleaned',
#                            f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train-google-fr-cleaned'])
FLAGS.train_input = f'{RECORDS_PATH}/{mark}/jigsaw-unintended-bias-train'
FLAGS.learning_rate=3e-5
FLAGS.opt_epsilon=1e-8
FLAGS.num_epochs=1
FLAGS.vie=0.1
FLAGS.do_test=1
run()

2020-04-25 00:05:19 0:00:28 -------Round: 0 mode: None train_input:[jigsaw-unintended-bias-train] valid_input:[validation] train_dirs:[1] valid_dir: ../input/tfrecords/xlm/validation
2020-04-25 00:05:19 0:00:28 --start_hour=jigsaw-unintended-bias-train --end_hour=validation root: ../input/tfrecords/xlm
2020-04-25 00:05:23 0:00:31 Model: "tf_roberta_model"
2020-04-25 00:05:23 0:00:31 _________________________________________________________________
2020-04-25 00:05:23 0:00:31 Layer (type)                 Output Shape              Param #   
2020-04-25 00:05:23 0:00:31 =================================================================
2020-04-25 00:05:23 0:00:31 roberta (TFRobertaMainLayer) ((None, 192, 768), (None, 278043648 
2020-04-25 00:05:23 0:00:31 =================================================================
2020-04-25 00:05:23 0:00:31 Total params: 278,043,648
2020-04-25 00:05:23 0:00:31 Trainable params: 278,043,648
2020-04-25 00:05:23 0:00:31 Non-trainable params: 0
2020-04-

Train for 7431 steps
   1/7431 [..............................] - ETA: 462:46:59 - loss: 0.9352 - acc: 0.0039 - auc: 0.4650

2020-04-25 00:09:10 0:04:19 Model total training parameters is: 278044417 with initial l2: 0.01613772557785255


 742/7431 [=>............................] - ETA: 2:40:56 - loss: 0.3334 - acc: 0.6628 - auc: 0.6782

eval: 100%|##########| 32/32 [00:00<00:00, 473.85it/s]

...

32/32 [==============================] - 39s 1s/step


2020-04-25 02:37:52 2:33:00 preds:
 +------+--------+----------+------------+
|   id | lang   |   y_true |       pred |
|------+--------+----------+------------|
| 2265 | tr     |        0 | 0.0240203  |
| 4993 | tr     |        0 | 0.00633908 |
| 1826 | it     |        0 | 0.0199839  |
| 2883 | es     |        0 | 0.00933557 |
| 5608 | tr     |        0 | 0.0058497  |
| 1921 | tr     |        1 | 0.0529662  |
| 2442 | it     |        1 | 0.108366   |
| 6235 | it     |        1 | 0.259334   |
| 1590 | es     |        1 | 0.294917   |
| 6659 | it     |        1 | 0.114269   |
+------+--------+----------+------------+
2020-04-25 02:37:52 2:33:00 eval_step: 10 step: 7430 epoch: 1.0
2020-04-25 02:37:52 2:33:00 valid_metrics:
 +---------+-----------+----------+----------+----------+---------+
|    loss |   auc/all |   auc/es |   auc/it |   auc/tr |   insts |
|---------+-----------+----------+----------+----------+---------|
| 0.32273 |  0.894193 | 0.871997 | 0.827132 | 0.974209 |    8000 |


7431/7431 [==============================] - 9147s 1s/step - loss: 0.2421 - acc: 0.6842 - auc: 0.8891


test: 100%|##########| 250/250 [00:00<00:00, 576.09it/s]


250/250 [==============================] - 97s 387ms/step


In [17]:
# odir = '../working/exps/toxic-mix-lang'
# os.system(f'mkdir -p f{odir}')
# model.save_weights(f'{odir}/model.h5')

In [18]:
df = pd.read_csv(f'{FLAGS.model_dir}/infos/test_1.csv')
df.to_csv('./submission.csv', index=False)
df.head()

,id,toxic
0,0,0.020160
1,1,0.004429
2,2,0.078021
3,3,0.005641
4,4,0.005713


In [19]:
# import traceback
# try:
#   optimizer = gezi.get('optimizer')
#   ckpt = tf.train.Checkpoint(model=model, optimizer=optimizer)
#   manager = tf.train.CheckpointManager(ckpt, FLAGS.model_dir, max_to_keep=1, checkpoint_name='model.ckpt')
#   manager.save()
#   model.save_weights(f'{FLAGS.model_dir}/model_weight.h5')
# except Exception:
#   print(traceback.format_exc())

In [20]:
os.system(f'mkdir -p {FLAGS.model_dir}/tf-xlm-roberta-base')
model.layers[-1].layers[0].save_pretrained(f'{FLAGS.model_dir}/tf-xlm-roberta-base')

I0425 02:39:43.036449 140030697813824 configuration_utils.py:142] Configuration saved in ../working/exps/unintended/tf-xlm-roberta-base/config.json
I0425 02:39:45.432259 140030697813824 modeling_tf_utils.py:246] Model weights saved in ../working/exps/unintended/tf-xlm-roberta-base/tf_model.h5
